# Plagiarism Detection Model

Now that we've created training and test data, we are ready to define and train a model. Our goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features we provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.



It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, we should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, we can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this we can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or we can manually upload our files to this notebook using the upload icon in Jupyter Lab. Then we can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

##  Upload your training data to S3

Specify the `data_dir` where we've saved our `train.csv` file. Decide on a descriptive `prefix` that defines where our data will be uploaded in the default S3 bucket. Finally, create a pointer to our training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

We are expected to upload our entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory we created to save our features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism-detection'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-534915205250/plagiarism-detection


### Test cell

Test that our data has been successfully uploaded. The below cell prints out the items in our S3 bucket and will throw an error if it is empty.We should see the contents of our `data_dir` and perhaps some checkpoints. If we see any other files listed, then we may have some old model files that we can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism-detection/test.csv
plagiarism-detection/train.csv
Test passed!


---

# Modeling

Now that we've uploaded your training data, it's time to define and train a model!

The type of model we create is up to you. For a binary classification task, we can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to we to test out a variety of models and choose the best one. Our project will be graded on the accuracy of your final model. 
 
---

## Complete a training script 

To implement a custom classifier, we'll need to complete a `train.py` script. We've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of our work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, we can use `!pygmentize` to display an existing `train.py` file. 



In [5]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib

## TODO: Import any additional libraries you need to define a model
from sklearn.svm import LinearSVC

# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
  

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function we specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [6]:

# your import and estimator code, here

from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point="train.py",
                    source_dir="source_sklearn",
                    role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge')

## EXERCISE: Train the estimator

Train our estimator on the training data stored in S3. This should create a training job that we can monitor in your SageMaker console.

In [7]:
%%time

# Train our estimator on S3 training data

estimator.fit({'train': input_data})

2020-04-02 09:28:46 Starting - Starting the training job...
2020-04-02 09:28:48 Starting - Launching requested ML instances......
2020-04-02 09:29:52 Starting - Preparing the instances for training...
2020-04-02 09:30:41 Downloading - Downloading input data...
2020-04-02 09:31:11 Training - Downloading the training image...
2020-04-02 09:31:47 Uploading - Uploading generated training model2020-04-02 09:31:32,200 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-02 09:31:32,202 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-02 09:31:32,212 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-02 09:31:39,860 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-04-02 09:31:39,861 sagemaker-containers INFO     Generating setup.cfg
2020-04-02 09:31:39,861 sagemaker-containers INFO     Generating MANIFEST.in
2020-04-02 09:31:39,861 sag

##  Deploy the trained model

After training, deploy our model to create a `predictor`. If we're using a PyTorch model, we'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, we'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [8]:
%%time



# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


-----------------!CPU times: user 269 ms, sys: 25.2 ms, total: 294 ms
Wall time: 8min 32s


---
# Evaluating Your Model

Once our model is deployed, we can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## Determine the accuracy of your model

Use our deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. We may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.



In [10]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [11]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test_y, test_y_preds)
print(accuracy)



## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**: 
* In my model prediction there are no false positive and false negatives. I got 100 % accuracy. The reason for that is we don't have a very big dataset with multiple features.I've tried few classifiers like (RandomForest) but i didn't 100 percent accuracy with that I got over 0.9. SVM are best classifiers when we have lesser number of features.Because of that I my Linear SVM model got 100% accuracy. 


In [12]:
df = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y), pd.DataFrame(test_y_preds)], axis=1)
df.columns=['c_1', 'c_5', 'lcs_word', 'class', 'predicted']
df

,c_1,c_5,lcs_word,class,predicted
0,1.000000,0.922280,0.820755,1,1
1,0.765306,0.589655,0.621711,1,1
2,0.884444,0.180995,0.597458,1,1
3,0.619048,0.043243,0.427835,1,1
4,0.920000,0.394366,0.775000,1,1
5,0.992674,0.973978,0.993056,1,1
6,0.412698,0.000000,0.346667,0,0
7,0.462687,0.000000,0.189320,0,0
8,0.581152,0.000000,0.247423,0,0
9,0.584211,0.000000,0.294416,0,0


### Question 2: How did you decide on the type of model to use? 

**Answer**:

Our main task is find the given text is plagiarised or not. That means it's a binary classification problem. And also it have lesser number of features. A Linear Support Vector Classification is a very good choice because it produces clear 1 or 0 instead of probabilities like Logistic Regression. It can also scale to a bigger number of samples.Because of that I have choosen that Linear SVM model for that task. It performed very well.I got 100% accuracy.



----
##  Clean up Resources

After we're done evaluating our model, **delete your model endpoint**. We can do this with a call to `.delete_endpoint()`. We need to show, in this notebook, that the endpoint was deleted. Any other resources, we may delete from the AWS console, and we will find more instructions on cleaning up all our resources, below.

In [13]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()


### Deleting S3 bucket

When we are *completely* done with training and testing models, we can also delete our entire S3 bucket. If we do this before we are done training your model, you'll have to recreate our S3 bucket and upload our training data again.

In [14]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '02638DB785D59FBE',
   'HostId': 'n912YoVPiPLAaKTJIkmYHWA3k1/YvXUqimYOSFbH5KaZvXFyp2TYvhywCnPYcDk7n2xn4flOBUg=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'n912YoVPiPLAaKTJIkmYHWA3k1/YvXUqimYOSFbH5KaZvXFyp2TYvhywCnPYcDk7n2xn4flOBUg=',
    'x-amz-request-id': '02638DB785D59FBE',
    'date': 'Thu, 02 Apr 2020 09:55:47 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism-detection/train.csv'},
   {'Key': 'plagiarism-detection/test.csv'},
   {'Key': 'sagemaker-scikit-learn-2020-04-02-09-28-46-207/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker-scikit-learn-2020-04-02-09-28-46-207/output/model.tar.gz'},
   {'Key': 'sagemaker-scikit-learn-2020-04-02-09-28-46-207/source/sourcedir.tar.gz'}]}]

### Deleting all your models and instances

When we are _completely_ done with this project and do **not** ever want to revisit this notebook, we can choose to delete all of our SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before we delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

